<a id='pd'></a>  
# Pandas DataFrames

[Create](#pd-create)  
[Import](#pd-import)  
[Restructure](#pd-restructure)  
[Explore](#pd-explore)  
[Missing Data](#pd-missing)  
[Transform](#pd-transform)  
[Select](#pd-select)  
[Filter](#pd-filter)  
[Query](#pd-query)  
[Aggregate](#pd-aggregate)  
[Resample](#pd-resample)  
[Visualize](#pd-visualize)   
[Export](#pd-export)  

## Initialize

### libraries

In [1]:
import os
import calendar
import warnings
import requests

#import pooch

import numpy as np

import pandas as pd 
from pandas.api.types import CategoricalDtype
#pd.__version__

import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib import colors
import seaborn as sns

In [2]:
# shorter traceback error messages
from functools import partial
get_ipython().showtraceback = partial(get_ipython().showtraceback,exception_only=True)

### directories

In [3]:
home_dir = home_dir = os.path.expanduser("~")
work_dir = os.path.join(home_dir, 'kr', 'df')
data_dir = os.path.join(home_dir, 'kr', 'df', 'data')
save_dir = os.path.join(home_dir, 'kr', 'df', 'save')

In [4]:
def create_directory(directory_name):
    if not os.path.exists(directory_name):
        os.mkdir(directory_name)

In [5]:
create_directory(work_dir)
create_directory(data_dir)
create_directory(save_dir)

In [6]:

os.chdir(work_dir)
os.getcwd()

'/Users/rkforest/kr/df'

### parameters

In [7]:
warnings.filterwarnings('ignore')

In [8]:
# shows result of cell without needing print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

In [9]:
sns.set(font_scale=1.5, style="darkgrid")

In [10]:
plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['figure.dpi'] = 144
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

In [11]:
pd.set_option('display.max_rows',12)
#defaults: pd.set_option('display.max_columns',20,'display.max_rows',60,'display.max_colwidth',50)

### constants

In [12]:
months = [month for month in calendar.month_abbr if month]
months_cat = CategoricalDtype(months, ordered=True)

seasons = ['Spring', 'Summer', 'Autumn','Winter']
seasons_cat = CategoricalDtype(seasons, ordered=True)

CategoricalDtype(categories=['Spring', 'Summer', 'Autumn', 'Winter'], ordered=True)

In [13]:
# create dictionary to assign seasons to months
months_season_dict = {}
for i, month in enumerate(months):
    if i in (0, 1, 11):
        months_season_dict[month] = 'Winter'
    elif i in (2, 3, 4):
        months_season_dict[month] = 'Spring'
    elif i in (5, 6, 7):
        months_season_dict[month] = 'Summer'        
    elif i in (8, 9, 10):
        months_season_dict[month] = 'Autumn' 

### palettes

In [14]:
seasons_colors = ['orange','green','darkred','blue']
seasons_cmap = ListedColormap(seasons_colors)
print(seasons_cmap)

In [15]:
seaborn_viridis_palette = sns.color_palette("viridis_r",4)
seaborn_viridis_palette;

In [16]:
reds_color_palette = sns.color_palette("Reds",18);

In [17]:
anomaly_color_palette = reds_color_palette[6:];

<a id='pd-create'></a>
## Create

[Return to Start of Notebook](#pd)  

### series

### dataframe

<a id='pd-import'></a>
## Import

[Return to Start of Notebook](#pd)  

- skiprows  
- sep  
- usecols  
- parse_dates  
- index_col (can also use integer of column)
- na_values  
- nrows  

### url

In [18]:
giss_url = "https://data.giss.nasa.gov/gistemp/tabledata_v4/"

'https://data.giss.nasa.gov/gistemp/tabledata_v4/'

### global monthly

In [19]:
file_id = "GLB"
file_name = file_id + ".Ts+dSST.csv"
file_url = giss_url + file_name
file_path = os.path.join(data_dir, file_name)
file_path

'/Users/rkforest/kr/df/data/GLB.Ts+dSST.csv'

In [20]:
file_path = os.path.join(data_dir, file_name)
file_path

'/Users/rkforest/kr/df/data/GLB.Ts+dSST.csv'

In [21]:
# download file_url and save to file_path
r = requests.get(file_url)
open(file_path, 'wb').write(r.content)

12518

In [22]:
dfg = pd.read_csv(file_path, skiprows=1) # initial read for columns
dfg.head(1)

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.18,-0.24,-0.08,-0.16,-0.09,-0.21,-0.18,-0.1,-0.14,-0.23,-0.21,-0.17,-0.16,***,***,-0.11,-0.16,-0.19


In [23]:
cols_to_import = dfg.columns[0:13]; # start to end+1
dfg = pd.read_csv(file_path,
                  skiprows=1,
                  sep=',',
                  usecols=cols_to_import,
                  parse_dates=['Year'],
                  #index_col=0, # can use either  = 0, = 'Year'
                  na_values=['***'],
                  nrows=150)
dfg = dfg.rename(columns={'Year': 'Date'})
dfg['Zone'] = file_id
dfg.head(1)

,Date,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Zone
0,1880-01-01,-0.18,-0.24,-0.08,-0.16,-0.09,-0.21,-0.18,-0.1,-0.14,-0.23,-0.21,-0.17,GLB


### northern hemisphere monthly

In [24]:
file_id = "NH"
file_name = file_id + ".Ts+dSST.csv"
file_url = giss_url + file_name
file_path = os.path.join(data_dir, file_name)
file_path


'/Users/rkforest/kr/df/data/NH.Ts+dSST.csv'

In [25]:
# download file_url and save to file_path
r = requests.get(file_url)
open(file_path, 'wb').write(r.content)

12561

In [26]:
dfn = pd.read_csv(file_path, skiprows=1) # initial read for columns
dfn.head(1)

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.36,-0.51,-0.23,-0.3,-0.06,-0.16,-0.19,-0.27,-0.24,-0.32,-0.43,-0.4,-0.29,***,***,-0.2,-0.21,-0.33


In [27]:
cols_to_import = dfn.columns[0:13]; # start to end+1
dfn = pd.read_csv(file_path,
                  skiprows=1,
                  sep=',',
                  usecols=cols_to_import,
                  parse_dates=['Year'],
                  #index_col=0, # can use either  = 0, = 'Year'
                  na_values=['***'],
                  nrows=150)
dfn = dfn.rename(columns={'Year': 'Date'})
dfn['Zone'] = file_id
dfn.head(1)

,Date,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Zone
0,1880-01-01,-0.36,-0.51,-0.23,-0.3,-0.06,-0.16,-0.19,-0.27,-0.24,-0.32,-0.43,-0.4,NH


### southern hemisphere monthly

In [28]:
file_id = "SH"
file_name = file_id + ".Ts+dSST.csv"
file_url = giss_url + file_name
file_path = os.path.join(data_dir, file_name)
file_path

'/Users/rkforest/kr/df/data/SH.Ts+dSST.csv'

In [29]:
# download file_url and save to file_path
r = requests.get(file_url)
open(file_path, 'wb').write(r.content)

12545

In [30]:
dfs = pd.read_csv(file_path, skiprows=1) # initial read for columns
dfs.head(1)

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,0.01,0.04,0.07,-0.01,-0.12,-0.24,-0.16,0.07,-0.04,-0.14,0.01,0.06,-0.04,***,***,-0.02,-0.11,-0.06


In [31]:

cols_to_import = dfs.columns[0:13]; # start to end+1
dfs = pd.read_csv(file_path,
                  skiprows=1,
                  sep=',',
                  usecols=cols_to_import,
                  parse_dates=['Year'],
                  #index_col=0, # can use either  = 0, = 'Year'
                  na_values=['***'],
                  nrows=150)
dfs = dfs.rename(columns={'Year': 'Date'})
dfs['Zone'] = file_id
dfs.head(1)

,Date,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Zone
0,1880-01-01,0.01,0.04,0.07,-0.01,-0.12,-0.24,-0.16,0.07,-0.04,-0.14,0.01,0.06,SH


### zonal annual

In [32]:
file_name = 'ZonAnn.Ts+dSST.csv'
file_url = giss_url + file_name
file_path = os.path.join(data_dir, file_name)
file_path

'/Users/rkforest/kr/df/data/ZonAnn.Ts+dSST.csv'

In [33]:
# download file_url and save to file_path
r = requests.get(file_url)
open(file_path, 'wb').write(r.content)

9981

In [34]:
dfz = pd.read_csv(file_path, skiprows=0) # initial read for columns
dfz.head(1)

,Year,Glob,NHem,SHem,24N-90N,24S-24N,90S-24S,64N-90N,44N-64N,24N-44N,EQU-24N,24S-EQU,44S-24S,64S-44S,90S-64S
0,1880,-0.16,-0.29,-0.04,-0.38,-0.12,-0.01,-0.81,-0.49,-0.29,-0.15,-0.09,-0.03,0.05,0.66


In [35]:
cols_to_import=['Year',
                'EQU-24N','24N-44N','44N-64N','64N-90N',
                '24S-EQU','44S-24S','64S-44S','90S-64S']
dfz = pd.read_csv(file_path, usecols=cols_to_import)
dfz.head(1)

,Year,64N-90N,44N-64N,24N-44N,EQU-24N,24S-EQU,44S-24S,64S-44S,90S-64S
0,1880,-0.81,-0.49,-0.29,-0.15,-0.09,-0.03,0.05,0.66


<a id='pd-restructure'></a>
## Restructure

[Return to Start of Notebook](#pd)  

### .melt

In [36]:
dfgt = pd.melt(dfg,
               id_vars=['Zone','Date'],
               var_name=['Mth'],
               value_name='Temp')
dfgt.head(1)

,Zone,Date,Mth,Temp
0,GLB,1880-01-01,Jan,-0.18


In [37]:
dfnt = pd.melt(dfn,
               id_vars=['Zone','Date'],
               var_name=['Mth'],
               value_name='Temp')
dfnt.head(1)

,Zone,Date,Mth,Temp
0,NH,1880-01-01,Jan,-0.36


In [38]:
dfst = pd.melt(dfs,
               id_vars=['Zone','Date'],
               var_name=['Mth'],
               value_name='Temp')
dfst.head(1)

,Zone,Date,Mth,Temp
0,SH,1880-01-01,Jan,0.01


In [39]:
dfzt = pd.melt(dfz,
             id_vars=['Year'],
             var_name=['Zone'],
             value_name='Temp')
dfzt.head(1)

,Year,Zone,Temp
0,1880,64N-90N,-0.81


### create year and month columns from date column

In [40]:
dfgt['Year'] = dfgt['Date'].dt.year
dfgt['Month'] = dfgt['Date'].dt.month

In [41]:
dfnt['Year'] = dfnt['Date'].dt.year
dfnt['Month'] = dfnt['Date'].dt.month

In [42]:
dfst['Year'] = dfst['Date'].dt.year
dfst['Month'] = dfst['Date'].dt.month

### reorder columns

In [43]:
dfgt = dfgt.reindex(['Date','Year','Month','Mth','Zone','Temp'], axis=1)
dfgt.columns

Index(['Date', 'Year', 'Month', 'Mth', 'Zone', 'Temp'], dtype='object')

In [44]:
dfnt = dfnt.reindex(['Date','Year','Month','Mth','Zone','Temp'], axis=1)
dfnt.columns

Index(['Date', 'Year', 'Month', 'Mth', 'Zone', 'Temp'], dtype='object')

In [45]:
dfst = dfst.reindex(['Date','Year','Month','Mth','Zone','Temp'], axis=1)
dfst.columns

Index(['Date', 'Year', 'Month', 'Mth', 'Zone', 'Temp'], dtype='object')

### save df as csv for importing to database

In [46]:
save_path = os.path.join(save_dir, 'global_temperature_anomalies.csv')
dfgt.to_csv(save_path, header=True, index=True, sep=',')

In [47]:
save_path = os.path.join(save_dir, 'northern_temperature_anomalies.csv')
dfnt.to_csv(save_path, header=True, index=True, sep=',')

In [48]:
save_path = os.path.join(save_dir, 'southern_temperature_anomalies.csv')
dfst.to_csv(save_path, header=True, index=True, sep=',')

### .pivot

In [ ]:
dfgp = dfgt.pivot(index='Year',
                  columns='Month',
                  values='Anomaly')
dfgp.head(1)

### .concat

#### stack vertically

In [ ]:
dfgt.shape

In [ ]:
dfm = pd.concat([dfgt,dfnt,dfst])
dfm.shape

#### stack horizontally

In [ ]:
df1 = dfnt.loc[:,['Year','Anomaly']]
df1.rename(columns={'Anomaly':'N_Anomaly'},inplace=True)
df2 = dfst.loc[:,['Anomaly']]
df2.rename(columns={'Anomaly':'S_Anomaly'},inplace=True)

pd.concat([df1,df2],axis=1).head(1) # stack horizontally

### .merge

- only joins dfs horizontally
- only exactly two dfs
- has options for inner, left, right, outer, cross
- can use any columns to match the dfs

In [ ]:
dfj = dfnt.merge(dfst,
                 how='inner',
                 on = ['Year','Month'])
dfj.head(3)

In [ ]:
dfj.shape

In [ ]:
dfj = dfnt.merge(dfst,
                 how='left',
                 on = ['Year','Month'],
                 indicator=True)
dfj.head(3)

In [ ]:
# using common index
df1 = dfnt.set_index('Year')
df2 = dfst.set_index('Year')
j = df1.merge(df2, how='inner', left_index=True, right_index=True)
j.head(3)

<a id='pd-explore'></a>
## Explore

[Return to Start of Notebook](#pd)   

Minimally sufficient pandas:  
- attributes  
  - shape, dtypes, columns, index, values  
- methods
  - isna, notna, idxmax, idxmin, nunique, value_counts
- other 
  - info, len, unique

### .info
df.info(memory_usage='deep')

In [ ]:
dfgt.info()

### .shape

In [ ]:
dfgt.shape

In [ ]:
nrows = dfgt.shape[0]
ncols = dfgt.shape[1]
print(nrows)
print(ncols)

### len()

In [ ]:
nrows = len(dfgt)

### .dtypes
returns series

In [ ]:
dfgt.dtypes

### .columns

In [ ]:
cols = dfgt.columns

### .index 

#### index type

In [ ]:
type(dfgt.index)

#### index name

In [ ]:
dfgt.index.name

#### index min, max

In [ ]:
x = dfgt.index.min()
y = dfgt.index.max()
print(x,y)

### .values

- pandas has separate numpy arrays for each data type in the df  
- when you access the values attribute, it concatenates them into a single array

#### values type

values returns a numpy array which can be used to improve performance when selecting single values

In [ ]:
type(dfgt.values)

#### values array

In [ ]:
dfgt.values

### .nunique
(dropna=False) to include Nan in total

In [ ]:
dfgt.nunique()

### .unique  
only available to series, not df

In [ ]:
unique_months = dfgt['Month'].unique()

### .value_counts  
The value_counts method is one of the most valuable methods for string columns.  
`df.value_counts(['col1','col2'],ascending=False,normalize=True).round(n)`

In [ ]:
vc = dfgt.value_counts('Month', ascending=True)
#vc = dfgt['Month'].value_counts(sort=True,normalize=True).round(3) # normalize
#vc = dfgt['Month'].value_counts().index[0]; # most frequent value

### .describe

In [ ]:
dfgt.describe()
#dfgt.describe().T # transpose with .T
#dfgt.describe(percentiles=[.1, .2, .5, .8, .9, .99]).T
#dfgt.describe(include='object')

### .corr

### .highlight
requires dataframe, not series: use [[ ]] 

In [ ]:
dfgt[['Anomaly']].head(5).style.format() \
    .highlight_max(color='yellow') \
    .highlight_min(color='lightblue')

<a id='pd-missing'></a>
## Missing data

[Return to Start of Notebook](#pd)  

### .isna.sum()

In [ ]:
dfgc = dfgt.copy()
dfgt.isna().sum().head(5) # count by column

#dfgt.isna().sum().sum() # total in df
#dfgt.isna().sum(axis=1) # count by row (sort first)

### .isna.any()

In [ ]:
filt = dfgt.isna().any(axis=1)
na_recs = dfgt.loc[filt]
na_recs.head(3)

### .notna

In [ ]:
filt = dfgt.notna()
sel = dfgt[filt]
sel.tail(3)

### .dropna

In [ ]:
dfgt.dropna(inplace=True) # use axis=1 to drop columns
dfgp.dropna(thresh=3); # row must have at least 4 not null values

### .fillna

#### specify fill value

In [ ]:
fill_value = round(dfgc.median(numeric_only=True),2) # median
fill_value = round(dfgc.mean(numeric_only=True),2) # mean
fill_value = 0 # constant
df_filled = dfgc.fillna(fill_value) 
df_filled .iloc[-1]

#### use dictionary

In [ ]:
df_filled = dfgc.fillna({'Anomaly': fill_value}) # use for multiple columns and values
df_filled .iloc[-1]

#### ffill and bfill methods

In [ ]:
df_filled = dfgc.fillna(method='ffill') # use previous value
#df_filled = dfgc.fillna(method='bfill') # use following value
df_filled.tail(2)

#### interpolate

In [ ]:
count_na = dfgc.isna().sum().sum()

In [ ]:
dfgt.tail()

In [ ]:
dfgc['Anomaly']= dfgc['Anomaly'].interpolate('linear')
dfgc.tail()

<a id='pd-transform'></a>
## Transform

[Return to Start of Notebook](#pd)  

### .rename

### .map

In [ ]:
dfgt['Season'] = dfgt['Month'].map(months_season_dict)  
dfgt.head(3)

### .astype 
- .astype('float64')
- .astype('int64') # cannot hold nulls
- .astype('bool') # cannot hold nulls
- .astype('str')
- .astype(categorical)

In [ ]:
# convert month and season columns to categorical types
dfgt['Month'] = dfgt['Month'].astype(months_cat)
dfgt['Season'] = dfgt['Season'].astype(seasons_cat)
dfgt.head(3)

### .sort_values

In [ ]:
dfgt.sort_values(by=['Year','Month'],inplace=True)
dfgt.head(3)

### .diff

In [ ]:
dfgt['Anomaly'] \
    .diff() \
    .nlargest()

### .pct_change

### .cum max,min,sum

### .str accessor
- .str.lower(), .upper(), .capitalize()
- .str.len()
- .str.count('a')
- .str.replace('x','y')
- .str.split(',')
- .str[0] first char
- .str.contains('x') > 0 (filter)
- .str.startswith('x'), .endswith('x') (filter)

### .dt

In [ ]:
dfcp = dfgt.copy()
dfcp['lower'] = dfcp['Season'].str.lower()
dfcp['upper'] = dfcp['Season'].str.upper()
dfcp['capitalize'] = dfcp['Season'].str.capitalize()
dfcp['len'] = dfcp['Season'].str.len()
dfcp['s3'] = dfgt['Season'].str[0:3]
dfcp['m1'] = dfgt['Month'].str[0]
dfcp.head(1)

#### .dt accessor

In [ ]:
dfcp = dfgt.copy().reset_index()
dfcp['year'] = dfcp['Year'].dt.year
dfcp['month'] = dfcp['Year'].dt.month
dfcp['day'] = dfcp['Year'].dt.day
dfcp['month_name'] = dfcp['Year'].dt.month_name()
dfcp.head(1)

#### .dt.strftime

In [ ]:
dfgt['year_str'] = dfgt['Year'].dt.strftime('%Y') # create year string

In [ ]:
dfcp = dfgt.copy().reset_index()
dfcp['%b'] = dfcp['Year'].dt.strftime('%b')
dfcp['%Y'] = dfcp['Year'].dt.strftime('%Y')
dfcp['%y'] = dfcp['Year'].dt.strftime('%y')
dfcp['%m'] = dfcp['Year'].dt.strftime('%m')
dfcp['%d'] = dfcp['Year'].dt.strftime('%d')
dfcp.head(1)

#### .dt.to_period
- A period is a special data type unique to pandas (it don't exist in numpy) 
- it represents an entire period of time such as the entire month  
- this contrasts with datetimes which represent a single moment in time

In [ ]:
dfcp = dfgt.copy().reset_index()
dfcp['month-period'] = dfcp['Year'].dt.to_period('M')
dfcp['year-period'] = dfcp['Year'].dt.to_period('Y')
dfcp['year-period-start'] = dfcp['year-period'].dt.start_time
dfcp['year-period-end'] = dfcp['year-period'].dt.end_time
dfcp.dtypes
dfcp.head(1)

### .to_datetime

In [ ]:
dfgt['month_str'] = pd.to_datetime(dfgt['Month'], format='%b').dt.strftime('%m') # create month string

In [ ]:
dfgt['date'] = pd.to_datetime(dfgt['year_str'] + dfgt['month_str'], format='%Y%m') # combine strings and convert to date
dfgt.head(1)

### .date_range

### .index

#### .set_index

In [ ]:
dfgt.set_index('date',inplace=True, verify_integrity=True, drop=True)
dfgt.head(1)

#### .index.names=

In [ ]:
dfgt.index.names = ['Date']
dfgt.head(1)

#### .reset_index

In [ ]:
dfgp.reset_index(inplace=True)
dfgp.head(1)

In [ ]:
dfgp.set_index('Year',inplace=True,drop=True)
dfgp.head(1)

#### .index. date accessor
(cannot use .dt)

In [ ]:
dfgt['Year'] = dfgt.index.year # add Year column
dfgt.head(1)

#### .sort_index

In [ ]:
#dfgt.sort_index(ascending=False,inplace=True)
dfgt.sort_index(inplace=True)
dfgt.head(3)

### .rank
- returns the ranking of each column, does not sort
- methods to handle ties:
  - average - default, ranks are average of ties
  - dense - ties given same rank, no numbers skipped
  - min - give each tie the minimum rank integer
  - max - give each tie the maximum rank number
  - first - arbitrarily give the tie that comes first in the dataset the lower/higher number.

In [ ]:
dfcp = dfgt.copy()
dfcp[['Anomaly']].rank().tail(3)
dfcp[['Anomaly']].rank(ascending=False).tail(3)
dfcp[['Anomaly']].rank(ascending=False, method='dense').tail(3)

### .drop_duplicates
- returns unique values 

In [ ]:
dfgc = dfgt.copy()
dfgc = dfgc[['Season','Month']].drop_duplicates(keep='last') # keep = first, last, False
dfgc.head(3)

### .apply

In [ ]:
dfcp = dfgt.copy()
dfcp['anomaly_f'] =  dfcp['Anomaly'].apply(lambda x: x * 1.8) # by column
dfcp.head(1)

In [ ]:
dfcp = dfgp.copy()
dfcp['any_gt_1'] =  dfgp.apply(lambda x: any(x > 0), axis = 1) # by row
dfcp.head(2)

### .drop

#### rows

In [ ]:
dropindex = dfgp.index[-1]
dfgp.drop(index=dropindex, inplace=True);
dfgp.tail(1)

#### columns

In [ ]:
cols = ['year_str', 'month_str']
dfgt.drop(columns=cols,inplace=True) # drop unneeded columns
dfgt.head(3)

### columns

#### add column

In [ ]:
dfgt['Id'] = 'GLB'
dfgt.head(1)

#### reorder columns

In [ ]:
dfgt = dfgt[['Id','Year','Season','Month','Anomaly']];  # reorder columns
dfgt.head(3)

<a id='pd-select'></a>
## Select

[Return to Start of Notebook](#pd)  

- columns with brackets []
- by label  .loc
- by integer location  .iloc
- using slice notation :

### columns

In [ ]:
scol= 'Anomaly'
scol1= ['Anomaly']
scols = ['Month','Anomaly']

icol = 4
icol1 = [4]
icols = [3,4];

In [ ]:
selection = dfgt[scol] # single column as series (brackets)
selection = dfgt.iloc[:,icol] # single column as series (iloc)
selection = dfgt[scol1] # single column as df (brackets)
selection = dfgt.iloc[:,icol1] # single column as df (iloc)

selection = dfgt[scols] # multiple columns (brackets)
selection = dfgt.iloc[:,icols] # multiple columns (iloc)
selection = dfgt.iloc[:,3:5] # multiple columns (slice notation)

selection = dfgt.select_dtypes('float64') # by date type
selection = dfgt.select_dtypes('int64') # by date type
selection = dfgt.select_dtypes('number') # by date type
selection = dfgt.select_dtypes('object') # by date type
selection = dfgt.select_dtypes('category') # by date type

selection = dfgp.sample(2, axis='columns'); # random sammple

### rows

In [ ]:
srow = '1900-01-01'
srow1 = ['1900-01-01']
srows = ['1900-01-01','1910-01-01']

irow = 0
irow1 = [0]
irows = [0,1];

In [ ]:
selection = dfgt.head(10)
selection = dfgt.tail(10)

selection = dfgt.loc[srow] # single row as series (loc)
selection = dfgt.iloc[irow] # single row as series (iloc)
selection = dfgt.loc[srow1] # single row as df (loc)
selection = dfgt.iloc[icol1] # single row as df (iloc)
selection = dfgt.loc['1880'] # single row as df partial date specification

selection = dfgt.loc[srows] # multiple rows (loc)
selection = dfgt.iloc[irows] # multiple rows (iloc)
selection = dfgt.loc['2000':'2020'] # multiple rows (slice notation)
selection = dfgt.iloc[0:3] # multiple rows (slice notation)

selection = dfgt.sample(2) # random sample
selection = dfgt.sample(frac=.10); # random sample fraction of rows
selection = dfgt.sample(2, random_state = 12345) # random sample defined id
selection = dfgt.sample(2, replace=True); # random sample with replacement

### rows and columns

In [ ]:
selection = dfgt.loc[srows,scols] # loc
selection = dfgt.iloc[irows,icols] # iloc

selection = dfgp.loc[srows, 'Jan':'Mar'] # loc slice notation
selection = dfgt.loc['1900':'1902', scols] # loc slice notation
selection = dfgp.loc['1900':'1902', 'Jan':'Mar'] # loc slice notation

selection = dfgt.iloc[irows, 1:3]
selection = dfgt.iloc[0:3, icols]
selection = dfgt.iloc[0:3, 1:3]

selection = dfgt.iloc[:, icols]; # all rows with slice notation

### single value

In [ ]:
selection = dfgp.loc['1900','Jan'] # as series
selection = dfgt.iloc[-3,0] # as value
selection = dfgt.loc[srow1,scol1] # as df
selection = float(dfgp.loc['1900','Jan']) # as float
selection = dfgt.values[5,1] # numpy values for improved performance

### using date index

In [ ]:
#selection = dfgt['2018-01-01':'2019-01-01']
selection = dfgt['2018':'2019']
selection.head(3)

<a id='pd-filter'></a>
## Filter

[Return to Start of Notebook](#pd)  

* Process of selecting rows based on values, not labels or integer locations.
* also known as boolean selection and boolean indexing  
* pandas defaults to boolean selection if contents of brackets is list of booleans
* operators  >, <, ==, !=, >=, <=  
* if also selecting specific columns, must use .loc
* boolean selection does not work with .iloc

### using date index

In [ ]:
filt = dfgt.index == '2018-01-01'
filt = dfgt.index > '2018-01-01'
filt = dfgt.index.year == 1990
dfgt[filt].head(3)

### using column values

In [ ]:
filt = dfgt["Anomaly"] >= 1.0
selection = dfgt.loc[filt].head(3)

In [ ]:
filt = dfgt["Season"]=="Winter"
winter_temps = dfgt[['Month','Anomaly']].loc[filt]
winter_temps.head(3)

In [ ]:
filt = dfgp['Jan'] > dfgp['Mar']
dfgp[['Jan','Mar']][filt].head(3)

### multiple conditions

In [ ]:
filt1 = dfgt["Anomaly"] > 1.1
filt2 = dfgt["Anomaly"] < 1.2
selection = dfgt.loc[filt1 & filt2].head(3) # and
#selection = dfgp.loc[filt1 | filt2] # or
#selection = dfgp.loc[~filt1]; # not

### between

In [ ]:
filt3 = dfgp["Jan"].between(1.02,1.2)
filt3 = dfgp["Jan"].between(1.02,2,inclusive='both')
filt3 = dfgp["Jan"].between(1.02,2,inclusive='neither')
selection = dfgp.loc[filt3]; # between

### isin

In [ ]:
filt = dfgt.index.year.isin([2020,2020])
selection = dfgt.loc[filt].head(2);

### count boolean values

In [ ]:
filt = dfgp['Jan'] > 1.0
count_true = sum(filt) # count

### average boolean values

In [ ]:
filt.mean() * 100 # as percent

### .idxmax, .idxmin

In [ ]:
imax = dfgt['Anomaly'].idxmax()
dfgt.loc[[imax],:]

In [ ]:
imax = dfgt['Anomaly'].idxmax()
imin = dfgt['Anomaly'].idxmin()
max_min_idx = [imax, imin]
dfgt.loc[max_min_idx,:]

### .nlargest, .nsmallest  
- keep = 'first' is default  
- keep = 'last' will instead keep last  
- keep = 'all' wiil keep all tied values

In [ ]:
selection = dfgt['Anomaly'].nlargest(n=3) # as series
#selection = dfgt['Anomaly'].nsmallest(n=3) # as series
#selection = dfgt.nlargest(3, 'Anomaly') # as df
#selection = dfgt['Anomaly'].nlargest(n = 3, keep = 'all') # keep ties

### chaining

In [ ]:
# max for each season
months_sorted = dfgt[['Season','Anomaly']] \
    .sort_values('Anomaly',ascending=False) \
    .drop_duplicates(subset=['Season'])

In [ ]:
dfgt['Anomaly'].diff().nlargest()

<a id='pd-query'></a>
## Query

[Return to Start of Notebook](#pd)  

- recommended over boolean selection
- however, does not cover all options that boolean selection has

In [ ]:
sel = dfgp.query('Jan > 1')
sel = dfgp.query('Jan > 1 and Feb < 1')
sel = dfgp.query('Jan > 1.2 or Feb > 1.2')
sel = dfgp.query('1 < Jan < 1.1')
sel = dfgp.query('Jan > Mar').head(3) # compare columns
sel = dfgp.query('Year == 2000') # uses year from Datetime index
sel = dfgp.query('Year in ["1880-01-01", "1980-01-01"]')

### @variable

In [ ]:
max_val = 1.0
sel = dfgp.query('Jan > @max_val')

### specify columns

In [ ]:
cols = ['Apr','Jun']
sel = dfgp.query('Jan > 1')[cols].head(3)

<a id='pd-aggregate'></a>
## Aggregate

[Return to Start of Notebook](#pd)  

### .stat
- sum, mean, median, min, max, count, std, var, describe, quantile

In [ ]:
dfgt['Anomaly'].mean()
dfgt.mean(numeric_only=True)

#### .agg

In [ ]:
month_stats = dfgt[['Anomaly']].agg(['sum','mean','max','min','median','count'],axis=0)
month_stats

#### axis

In [ ]:
annual_stats = dfgp.agg(['sum','mean','max','min','median','count'],axis=1,numeric_only=True)
annual_stats.head(3)

### .groupby()

#### split using .groupby()

In [ ]:
grouped_by_season = dfgt.groupby('Season')
type(grouped_by_season)

In [ ]:
for idx, grp in grouped_by_season:
    print(idx,len(grp),grp['Anomaly'].mean())

#### apply aggregation to groups
(results combined into new df)

In [ ]:
season_avg = grouped_by_season[['Anomaly']].mean()
print(type(season_avg))

#### chain as single step

In [ ]:
season_avg = dfgt.groupby('Season')[['Anomaly']].mean()
season_avg

### .groupby().agg()

#### tuple syntax

In [ ]:
grouped_by_season = dfgt.groupby('Season').agg(
    avg_anomaly=('Anomaly', 'mean'),
    max_anomaly=('Anomaly', 'max'),
    min_anomaly=('Anomaly', 'min'))
grouped_by_season

#### dictionary syntax

In [ ]:
grouped_by_season = dfgt.groupby('Season').agg(
    {'Anomaly': ['mean', 'max', 'min', 'count']})
grouped_by_season

##### .droplevel()

In [ ]:
grouped_by_season.columns = grouped_by_season.columns.droplevel(0)
grouped_by_season

#### brackets syntax

#### group with multiple columns
observed = True

In [ ]:
grouped_by_season = dfgt.groupby(['Season','Month'], observed=True)\
    .agg(avg_anomaly=('Anomaly', 'mean'),
         max_anomaly=('Anomaly', 'max'),
         count_anomaly=('Anomaly', 'count'))\
    .round(3)\
    .sort_values(by='avg_anomaly',ascending=False)\
    .reset_index()
grouped_by_season

#### apply function to group

In [ ]:
dfgt.groupby('Season')\
    .agg(season_diff=('Anomaly', lambda x: x.max() - x.min()),
         season_max=('Anomaly','max'),
         season_min=('Anomaly','min'))

In [ ]:
dfgt.groupby('Season')\
    .agg(avg_largest_5=('Anomaly', lambda x: x.nlargest(5).mean()),
         anomaly_max=('Anomaly','max'))

In [ ]:
grp = dfgt.groupby('Month')\
    .agg(pct_gt_1=('Anomaly', lambda x: (x>1).mean()*100))\
    .round(2)\
    .sort_values(by='pct_gt_1',ascending=False)    
grp.head(3)

### .groupby().filter()
- requires writing a custom function or using lambda
- function must return a single boolean value
- filters passes each group as a df
- each group kept or dropped based on returned boolean value
- end result is rows in original df that meet criteria

In [ ]:
# task: returns all rows in group where max anomaly of group > 1.35
# use this for checking results of groupby filter
grp = dfgt.groupby('Month')\
    .agg(max_anomaly=('Anomaly', 'max'))\
    .reset_index()\
    .sort_values(by='max_anomaly',ascending=False)
grp.head(3)

#### single step using .groupby().filter()

In [ ]:
grp = dfgt.groupby('Month')\
    .filter(lambda x: x['Anomaly'].max() > 1.35)\
    .round(2)
grp.head(3)

#### comparable steps without group filter

### .groupby().transform()
- similar to .agg()
- returns same number of rows as orginal df as a series
- syntax: df.groupby('grouping col')['transformed col'].transform(func)
- can use string name of pandas function or custom function or lambda

#### add group max to each row

In [ ]:
dfgtc = dfgt.copy()
dfgtc['month max'] = dfgtc.groupby('Month')['Anomaly'].transform('sum')
dfgtc.head(3)

#### add difference from group mean to each row

In [ ]:
dfgtc = dfgt.copy()
dfgtc['month mean'] = dfgtc.groupby('Month')['Anomaly'].transform('mean').round(2)
dfgtc['diff_mean'] = dfgtc.groupby('Month')['Anomaly'].transform(lambda x: x - x.mean()).round(2)
dfgtc.head(3)

### pivot table

In [ ]:
pt = dfgt.pivot_table(
    index='Season',
    columns='Month', 
    values='Anomaly',
    aggfunc=['mean'],
    fill_value=0,
    margins=True).round(3)
pt

### crosstab
adds ability to normalize (normalize options: index, columns, all)

In [ ]:
pd.crosstab(index=dfgt['Season'], 
            columns=dfgt['Month'], 
            normalize='all').round(3) * 100

### bins

In [ ]:
col_min, col_max = dfgt['Year'].agg(['min','max'])
print(col_min, col_max)

In [ ]:
dfc = dfgt.copy().reset_index()
bins_20yr = [col_min-1,1880,1900,1920,1940,1960,1980,2000,2020,col_max]
dfc['bins'] = pd.cut(dfc['Year'], bins = bins_20yr)
dfc.head(3)

In [ ]:
dfc.groupby('bins')['Anomaly'].agg(['mean','count'])

#### quantile binning .qcut()

In [ ]:
dfc = dfgt.copy().reset_index()
dfc['qbins'] =  pd.qcut(dfc['Year'], 5)
dfc.groupby('qbins')['Anomaly'].agg(['mean','count'])

### rolling

In [ ]:
dfgt['Anomaly'].rolling(5,min_periods=1).agg(['min', 'max', 'mean','count']).head()
#dfgt['Anomaly'].rolling(5,min_periods=1,center=True).agg(['min', 'max', 'mean','count']).head()

<a id='pd-resample'></a>
## Resample

[Return to Start of Notebook](#pd)  

- nnAS year start frequency
- nnYS year start frequency
- nnY year end frequency
- nnA year end frequency
- 10Y = decade

### .resample().agg()

In [ ]:
dfgty = dfgt[['Anomaly']]\
            .resample('YS')\
            .agg(avg_anomaly=('Anomaly', 'mean'),
                max_anomaly=('Anomaly', 'max'),
                min_anomaly=('Anomaly', 'min'),
                count_anomaly= ('Anomaly', 'count'))\
            .round(2)\
            .reset_index()\
            .set_index('Date')
dfgty.head(3)

### .groupby().resample().agg()

In [ ]:
dfgt10y = dfgt[['Id','Month','Season','Anomaly']]\
            .groupby(['Id','Month','Season'])\
            .resample('10YS')\
            .agg(avg_anomaly = ('Anomaly','mean'),\
                 count_values = ('Anomaly','count'))\
            .round(3)\
            .reset_index()\
            .set_index('Date')
dfgt10y.tail(3)

#### filter for complete decades

In [ ]:
filt = dfgt10y["count_values"]>=10
dfgt10y = dfgt10y.loc[filt]
dfgt10y.tail(3)

In [ ]:
dfgt10y['Decade'] = dfgt10y.index.year # add Decade column to decades df
dfgt10y = dfgt10y[['Id','Decade','Season','Month','avg_anomaly']] # reorder columns
dfgt10y.tail(3)

<a id='pd-visualize'></a>
## Visualize

[Return to Start of Notebook](#pd)  

### scatter plot

In [ ]:
divnorm = colors.TwoSlopeNorm(vmin=dfgt['Anomaly'].max()*-1,
                              vcenter=0.,
                              vmax=dfgt['Anomaly'].max())
g = sns.relplot(
    kind="scatter", height=4, aspect=2,
    data=dfgt, x='Year', y='Anomaly',hue='Anomaly', legend=False, palette="coolwarm", hue_norm=divnorm)
g.set(title="Global Temperature Anomalies", xlabel="", ylabel="Degrees (C)")

sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=divnorm)
for ax in g.axes.flat:
    ax.figure.colorbar(sm)

### line plot

In [ ]:
g = sns.relplot(
    kind="line", height=2, aspect=2,
    data=dfgt, x='Date', y='Anomaly',
    hue='Season', hue_order=seasons,
    palette=seaborn_viridis_palette,
    col='Season', col_wrap=2, legend=False)
g.set(xlabel='', ylabel = "Degrees (C)");
g.fig.suptitle('Global Temperature Anomalies', y=1.03);
for col_key, ax in g.axes_dict.items():
    ax.set_title(col_key)

### bar plot

In [ ]:
g = sns.catplot(
    kind="bar", height=4, aspect=1.5,
    data=dfgt, x='Season', y='Anomaly',
    ci=None, edgecolor = 'black', saturation=1.0, lw=0.5,
    order=seasons, palette=seaborn_viridis_palette, legend=False)
g.set(title = "Global Temperature Anomalies", xlabel="", ylabel = "Degrees (C)")

### boxplots

In [ ]:
dfgt.head()

In [ ]:
sns.set_style('darkgrid')

g = sns.catplot(
    kind="box", height=3, aspect=2,
    data=dfgt, y='Season', x='Anomaly',
    palette=seaborn_viridis_palette, saturation=1.0,
    )
g.set(title = "Global Temperature Anomalies", ylabel="", xlabel = "Degrees (C)")   

### histograms

In [ ]:
g = sns.displot(
    kind="hist", height=3, aspect=2,
    data=dfgt, x='Anomaly')
g.set(title = "Global Temperature Anomalies",  xlabel = "Degrees (C)")  

<a id='pd-export'></a>
## Export

[Return to Start of Notebook](#pd)  

In [ ]:
save_path = os.path.join(save_dir, 'global_temp_anomalies_10yr.csv')
dfgt10y.to_csv(save_path, header=True, index=True, sep=',')